<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [21]</a>'.</span>

# Filters and combines existing clusters based on tools used, start and end dates, and organization
## Rebuilds all clusters (will need to run daily)

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",   
} 
    #"username": "wang159-4j1v@force.com",
    #"password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Pull data from SF - this will be a post SF filter

In [3]:
##########################################################################################
## 1 - Pull all tool_usage_cluster__c information
##     i) ID
##     ii) Ending_Date__c
##     iii) Starting_Date__c
##     iv) Organization__c
##     v) Semester__c
## 2 - Pull all ContactToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Contact__c
## 3 - Pull all ToolToolClusterAssociation__c information
##     i) Tool_Usage_Cluster__c
##     ii) Tool__c
## 4 - determine which tool_usage_cluster__c are suspicious 
##     i) first, group tool_usage_cluster__c by tool__c used
##     ii) check starting and ending dates - if they overlap, make a note of the tool_usage_clusters
## 5 - combining tool_usage_cluster__c 
##     i) use one of the existing Ids, mark the others for deletion
##     ii) when combining, pull the relevant Tool__c information and Contact__c information
##
##########################################################################################

In [4]:
db_1 = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [5]:
tool_cluster_df = db_1.query_data('Select ID, Ending_Date__c, Starting_Date__c, Organization__c,Semester__c \
                        from tool_usage_cluster__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TpzBfAAJ
{"id":"7505w00000TpzBfAAJ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T18:54:45.000+0000","systemModstamp":"2021-02-09T18:54:45.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000TpzBfAAJ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T18:54:45.000+0000","systemModstamp":"2021-02-09T18:54:46.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":3854,"retries":0,"totalProcessingTime":632}
[Success] Bulk job completed successfully.


In [6]:
display(tool_cluster_df.head(5))

,Ending_Date__c,Id,Organization__c,Semester__c,Starting_Date__c
0,2011-01-30,a0w5w000009Q7CuAAK,NaN,Fall,2010-09-05
1,2018-11-30,a0w5w000009Q7CvAAK,a0r5w00000V42iyAAB,Fall,2018-11-29
2,2019-11-28,a0w5w000009Q7CwAAK,a0r5w00000V42iyAAB,Fall,2019-11-04
3,2017-12-16,a0w5w000009Q7CxAAK,a0r5w00000V42cCAAR,Fall,2017-10-03
4,2018-01-21,a0w5w000009Q7CyAAK,a0r5w00000V42cCAAR,Spring,2018-01-15


In [7]:
contact_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Contact__c from ContactToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TpzJdAAJ
{"id":"7505w00000TpzJdAAJ","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T18:54:59.000+0000","systemModstamp":"2021-02-09T18:55:00.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000TpzJdAAJ","operation":"query","object":"ContactToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T18:54:59.000+0000","systemModstamp":"2021-02-09T18:55:08.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":81853,"retries":0,"totalProcessingTime":6611}
[Success] Bulk job completed successfully.


In [23]:
contact_in_cluster_df.shape

(81853, 4)

In [8]:
display(contact_in_cluster_df.head(5))

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JwbeAAC,a0x5w00000c4aLPAAY,tejaswaghate_637,a0w5w000009Q7p0AAC
1,0035w000034JrYGAA0,a0x5w00000c4aLQAAY,paragbhandarkar_637,a0w5w000009Q7p0AAC
2,0035w000034JrYDAA0,a0x5w00000c4aLRAAY,anumolsasi69_637,a0w5w000009Q7p0AAC
3,0035w000034JmUnAAK,a0x5w00000c4aLSAAY,amitkumarsingh89_637,a0w5w000009Q7p0AAC
4,0035w000034JwP4AAK,a0x5w00000c4aLTAAY,seema.gumbade_637,a0w5w000009Q7p0AAC


In [9]:
tools_in_cluster_df = db_1.query_data('Select Id, Name, Tool_Usage_Cluster__c, Tool__c from ToolToolClusterAssociation__c')

[Success] Bulk job creation successful. Job ID = 7505w00000TpzKCAAZ
{"id":"7505w00000TpzKCAAZ","operation":"query","object":"ToolToolClusterAssociation__c","createdById":"0055w00000DM5bOAAT","createdDate":"2021-02-09T18:55:20.000+0000","systemModstamp":"2021-02-09T18:55:22.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":5850,"retries":0,"totalProcessingTime":740}
[Success] Bulk job completed successfully.


In [10]:
display(tools_in_cluster_df.head(5))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000N0Um8AAF,cntbands-ext_315,a0w5w00000A8OGBAA3,a0s5w00000k5MPqAAM
1,a0y5w00000N0Um9AAF,fettoy_315,a0w5w00000A8OGBAA3,a0s5w00000k5MQwAAM
2,a0y5w00000N0UmAAAV,pntoy_315,a0w5w00000A8OGBAA3,a0s5w00000k5MR0AAM
3,a0y5w00000N0UmBAAV,qdot_315,a0w5w00000A8OGBAA3,a0s5w00000k5MR2AAM
4,a0y5w00000N0UmCAAV,prophet_315,a0w5w00000A8OGBAA3,a0s5w00000k5MQzAAM


### determining suspicious tool_usage_clusters__c

In [11]:
## overarching goal: determine the cluster IDs that need further investigation

# grouping the clusters by tools used (i.e., clusters by tool)
tools_dict = {j: [] for j in tools_in_cluster_df['Tool__c'].to_list()} #stores relevant indexes per org

for i,j in enumerate(tools_in_cluster_df['Tool__c'].to_list()):
    tools_dict[j].append(i)

unique_tools = list(tools_dict.keys())

In [12]:
from copy import deepcopy

In [13]:
## check start and end dates for overlaps
overlaps = ['init'] # del index 0 later

cluster_ids = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
tool_cluster_id_indexes = tool_cluster_df['Id'].to_list()

for i,ii in enumerate(unique_tools):
    for j in tools_dict[ii]: # this requires a sequential comparison
        ## tools dict contains the tools_in_cluster_df index
        temp_set = deepcopy(tools_dict[ii])
        del temp_set[temp_set.index(j)]
        
        # pull id
        current_index = tool_cluster_id_indexes.index(cluster_ids[j]) #integer index
        
        current_org = tool_cluster_df.iloc[current_index,2]
        current_sem = tool_cluster_df.iloc[current_index,3]
        current_start = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,-1],'%Y-%m-%d')
        current_stop = datetime.datetime.strptime(tool_cluster_df.iloc[current_index,0],'%Y-%m-%d')
        
        # sequential compare
        for k in temp_set:
            temp_index = tool_cluster_id_indexes.index(cluster_ids[k]) #integer index
            
            temp_org = tool_cluster_df.iloc[temp_index,2]
            temp_sem = tool_cluster_df.iloc[temp_index,3]
            temp_start = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,-1],'%Y-%m-%d')
            temp_stop = datetime.datetime.strptime(tool_cluster_df.iloc[temp_index,0],'%Y-%m-%d')
        
            if temp_org == current_org and temp_sem == current_sem:
                # now check dates for overlap - if there is overlap, assume same cluster
                if temp_start > current_stop or temp_stop < current_start: #case 1 - () [], # case 2 - [] ()
                    # do nothing
                    filler = 0
                else:
                    # include the overlap as a tuple
                    overlaps.append([cluster_ids[j],cluster_ids[k]])


In [14]:
## filter the set of overlaps
if overlaps[0] == 'init':
    del overlaps[0]

for i,j in enumerate(overlaps):
    j.sort()
    overlaps[i] = j

overlaps.sort()

In [15]:
overlaps[:5]

[['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFhAAN']]

In [16]:
import itertools

In [17]:
overlaps2 = list(k for k,_ in itertools.groupby(overlaps))

In [18]:
overlaps2[:9]

[['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFhAAN'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFiAAN'],
 ['a0w5w000009Q7D9AAK', 'a0w5w00000A8OFkAAN'],
 ['a0w5w000009Q7D9AAK', 'a0w5w00000A8OFlAAN'],
 ['a0w5w000009Q7DFAA0', 'a0w5w00000A8OFnAAN'],
 ['a0w5w000009Q7DFAA0', 'a0w5w00000A8UFNAA3'],
 ['a0w5w000009Q7DMAA0', 'a0w5w00000A8OFoAAN']]

In [19]:
print(len(overlaps),len(overlaps2))

1510 612


#### now that we have the set of overlapping clusters, we need to specify a main one

In [20]:
## main cluster characteristics
## Part A
## 1. combine overlaps together (i.e., three overlapping clusters combined into 1) 
## 2. decide the main cluster
## Part B
## 1. start and end dates realignment - i.e., should take the min of all the start dates and max of all the ends
## 2. combine contacts
## 3. create two DataFrames, one for upsert and the other for deletion

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [21]:
## Part A
# collapse overlaps2
overlaps3 = ['init']
main_clusters = []

running_temp_init = True

for i,j in enumerate(overlaps2):
    if running_temp_init == True:
        main_clusters.append(j[0])
        overlaps_holder = deepcopy(j)
        running_temp_init = False
    else:
        if j[0] in overlaps_holder and j[1] not in overlaps_holder:
            overlaps_holder.append(j[1])
        else: 
            if j[0] not in overlaps_holder and j[1] not in overlaps_holder: #neither is in overlaps_holder
                overlaps3.append(overlaps_holder)

                # recalc
                main_clusters.append(j[0])
                overlaps_holder = deepcopy(j)

# final iteration
overlaps3.append(overlaps_holder)

In [22]:
len(main_clusters)

340

In [24]:
if overlaps3[0] == 'init':
    del overlaps3[0]
    
print(len(overlaps3))

340


In [25]:
display(main_clusters[:5])
display(overlaps3[:5])

['a0w5w000009Q7CwAAK',
 'a0w5w000009Q7CxAAK',
 'a0w5w000009Q7CzAAK',
 'a0w5w000009Q7D9AAK',
 'a0w5w000009Q7DFAA0']

[['a0w5w000009Q7CwAAK', 'a0w5w00000A8WKZAA3'],
 ['a0w5w000009Q7CxAAK', 'a0w5w00000A8OJTAA3'],
 ['a0w5w000009Q7CzAAK', 'a0w5w00000A8OFhAAN', 'a0w5w00000A8OFiAAN'],
 ['a0w5w000009Q7D9AAK', 'a0w5w00000A8OFkAAN', 'a0w5w00000A8OFlAAN'],
 ['a0w5w000009Q7DFAA0', 'a0w5w00000A8OFnAAN', 'a0w5w00000A8UFNAA3']]

### Major step - building upsert and deletion dataframes
### Build 3 upserts and 3 deletions [corresponds to tool_usage_cluster__c, ContactToolClusterAssociation__c, ToolToolClusterAssociation__c]

In [26]:
## Part B
### build the upsert dataframe first
upsert_df = pd.DataFrame(columns=['Id','Ending_Date__c','Starting_Date__c']) #Id of the main cluster
upsert_df['Id'] = deepcopy(main_clusters)

In [27]:
for i,ii in enumerate(main_clusters):
    main_c_index = tool_cluster_df['Id'].to_list()
    c_start_date = tool_cluster_df.iloc[main_c_index.index(ii),-1]    
    c_end_date = tool_cluster_df.iloc[main_c_index.index(ii),0]
        
    for j,jj in enumerate(overlaps3[i]):
        j_start_date = tool_cluster_df.iloc[main_c_index.index(jj),-1]
        j_end_date = tool_cluster_df.iloc[main_c_index.index(jj),0]
        
        if j_start_date < c_start_date:
            c_start_date = deepcopy(j_start_date)
        
        if j_end_date > c_end_date:
            c_end_date = deepcopy(j_end_date)
        
    upsert_df.iloc[i,1:] = [c_end_date,c_start_date]

In [28]:
display(upsert_df.head(5))
display(upsert_df.tail(5))

,Id,Ending_Date__c,Starting_Date__c
0,a0w5w000009Q7CwAAK,2019-11-28,2019-11-04
1,a0w5w000009Q7CxAAK,2017-12-16,2017-10-03
2,a0w5w000009Q7CzAAK,2018-04-19,2018-02-05
3,a0w5w000009Q7D9AAK,2007-04-23,2007-02-07
4,a0w5w000009Q7DFAA0,2008-12-18,2008-10-05


,Id,Ending_Date__c,Starting_Date__c
335,a0w5w00000A8OPJAA3,2013-10-05,2013-08-27
336,a0w5w00000A8OPMAA3,2015-10-17,2015-08-27
337,a0w5w00000A8OPPAA3,2018-04-17,2018-01-12
338,a0w5w00000A8OPeAAN,2020-08-04,2020-07-26
339,a0w5w00000A8OQ4AAN,2020-12-11,2020-10-13


In [29]:
### buliding the delete dataframe - should be all Ids in overlaps that are not main clusters
deletion_df = pd.DataFrame(columns=['Id']) #Id of the main cluster

deletion_ids = []

for i,ii in enumerate(main_clusters):
    for j in overlaps3[i][1:]:
        deletion_ids.append(j)
deletion_df['Id'] = deletion_ids

In [30]:
display(deletion_df.head(3))

,Id
0,a0w5w00000A8WKZAA3
1,a0w5w00000A8OJTAA3
2,a0w5w00000A8OFhAAN


In [31]:
## summary up to this point
## upsert_df db.send_data(upsert_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

## deletion_df db.delete_data(deletion_df)
## db.object_id = 'tool_usage_cluster__c'
## db.external_id = 'Id'

In [32]:
## updating contact and cluster mapping
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JwbeAAC,a0x5w00000c4aLPAAY,tejaswaghate_637,a0w5w000009Q7p0AAC
1,0035w000034JrYGAA0,a0x5w00000c4aLQAAY,paragbhandarkar_637,a0w5w000009Q7p0AAC


(81853, 4)


In [33]:
# NOTE: there are contacts that belong to non-overlapping clusters
contact_usage_clusters = contact_in_cluster_df['Tool_Usage_Cluster__c'].to_list()
contacts = contact_in_cluster_df['Contact__c'].to_list()
change_indexes = []

## check if current cluster belongs to overlaps3 - first need to convert overlaps3 into a direct list
overlaps4 = list(itertools.chain.from_iterable(overlaps3))

In [34]:
print(len(overlaps4))

889


In [35]:
## populate the contacts that need changing
for i,j in enumerate(contact_usage_clusters):
    if j in overlaps4:
        change_indexes.append(i)

In [36]:
print(len(change_indexes))

26643


In [37]:
## search for each index that needs changing
for i in change_indexes:
    if contact_usage_clusters[i] not in main_clusters:
        #print('testouter')
        
        # need to find its main_cluster
        found_main = False
        main_counter = 0
        while found_main == False:
            if contact_usage_clusters[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                contact_usage_clusters[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [38]:
contact_in_cluster_df['Tool_Usage_Cluster__c'] = contact_usage_clusters

In [39]:
## remove duplicates from contact_in_cluster_df
contact_in_cluster_df = contact_in_cluster_df.drop_duplicates()
display(contact_in_cluster_df.head(2))
print(contact_in_cluster_df.shape)

,Contact__c,Id,Name,Tool_Usage_Cluster__c
0,0035w000034JwbeAAC,a0x5w00000c4aLPAAY,tejaswaghate_637,a0w5w000009Q7p0AAC
1,0035w000034JrYGAA0,a0x5w00000c4aLQAAY,paragbhandarkar_637,a0w5w000009Q7p0AAC


(81853, 4)


In [40]:
## tools - take max, i.e. set all constituent clusters of a main cluster to the main cluster
## then remove duplicates
display(tools_in_cluster_df.head(2))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000N0Um8AAF,cntbands-ext_315,a0w5w00000A8OGBAA3,a0s5w00000k5MPqAAM
1,a0y5w00000N0Um9AAF,fettoy_315,a0w5w00000A8OGBAA3,a0s5w00000k5MQwAAM


In [41]:
tuc = tools_in_cluster_df['Tool_Usage_Cluster__c'].to_list()

# first find those cluster ids that are clustered
tool_cluster_ids = []
for i,j in enumerate(tuc):
    if j in overlaps4:
        tool_cluster_ids.append(i)

In [42]:
print(len(tool_cluster_ids))

1373


In [43]:
for i in tool_cluster_ids:
    if tuc[i] not in main_clusters: #then need to find its corresponding main cluster
        found_main = False
        main_counter = 0
        
        while found_main == False:
            if tuc[i] in overlaps3[main_counter]:
                #print('test')
                found_main = True
                tuc[i] = overlaps3[main_counter][0]
            else:
                main_counter += 1

In [44]:
tools_in_cluster_df['Tool_Usage_Cluster__c'] = tuc
display(tools_in_cluster_df.head(2))

,Id,Name,Tool_Usage_Cluster__c,Tool__c
0,a0y5w00000N0Um8AAF,cntbands-ext_315,a0w5w000009Q7jnAAC,a0s5w00000k5MPqAAM
1,a0y5w00000N0Um9AAF,fettoy_315,a0w5w000009Q7jnAAC,a0s5w00000k5MQwAAM


## Send to SF

In [45]:
db_upsert_del = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [46]:
db_upsert_del.object_id = 'tool_usage_cluster__c'
db_upsert_del.external_id = 'Id'

In [47]:
db_upsert_del.send_data(upsert_df)

[Success] Bulk job creation successful. Job ID = 7505w00000TpzM8AAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000TpzM8AAJ
[Success] Closing job successful. Job ID = 7505w00000TpzM8AAJ


In [48]:
db_upsert_del.check_bulk_status()

{'id': '7505w00000TpzM8AAJ',
 'operation': 'upsert',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-09T18:58:26.000+0000',
 'systemModstamp': '2021-02-09T18:58:29.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Id',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [49]:
from pprint import pprint
pprint(db_upsert_del.check_bulk_failed_results())

''


#### upload contact_in_cluster_df

In [50]:
db_contact_upsert = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [51]:
## need to delete first, then upload
contact_in_cluster_df_del = pd.DataFrame(contact_in_cluster_df['Id'])
contact_in_cluster_df_up = contact_in_cluster_df.drop(columns='Id')

In [52]:
display(contact_in_cluster_df_del.head(2))
display(contact_in_cluster_df_up.head(2))

,Id
0,a0x5w00000c4aLPAAY
1,a0x5w00000c4aLQAAY


,Contact__c,Name,Tool_Usage_Cluster__c
0,0035w000034JwbeAAC,tejaswaghate_637,a0w5w000009Q7p0AAC
1,0035w000034JrYGAA0,paragbhandarkar_637,a0w5w000009Q7p0AAC


In [53]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Id'

In [54]:
db_contact_upsert.delete_data(contact_in_cluster_df_del)

[Success] Bulk job creation successful. Job ID = 7505w00000TpzMIAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000TpzMIAAZ
[Success] Closing job successful. Job ID = 7505w00000TpzMIAAZ


In [55]:
db_contact_upsert.check_bulk_status()

{'id': '7505w00000TpzMIAAZ',
 'operation': 'delete',
 'object': 'ContactToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-09T18:58:34.000+0000',
 'systemModstamp': '2021-02-09T18:58:39.000+0000',
 'state': 'InProgress',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [56]:
pprint(db_contact_upsert.check_bulk_failed_results())

''


In [57]:
db_contact_upsert.object_id = 'ContactToolClusterAssociation__c'
db_contact_upsert.external_id = 'Name'

In [58]:
db_contact_upsert.send_data(contact_in_cluster_df_up)

[Success] Bulk job creation successful. Job ID = 7505w00000TpzMNAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000TpzMNAAZ
[Success] Closing job successful. Job ID = 7505w00000TpzMNAAZ


In [59]:
db_contact_upsert.check_bulk_status()

{'id': '7505w00000TpzMNAAZ',
 'operation': 'upsert',
 'object': 'ContactToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-09T18:58:42.000+0000',
 'systemModstamp': '2021-02-09T18:58:52.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Name',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [60]:
pprint(db_contact_upsert.check_bulk_failed_results())

''


#### tools_in_cluster_df

In [61]:
db_tool_upsert = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [62]:
## need to delete first, then upload
db_tool_upsert_del = pd.DataFrame(tools_in_cluster_df['Id'])
db_tool_upsert_up = tools_in_cluster_df.drop(columns='Id')

In [63]:
display(db_tool_upsert_del.head(2))
display(db_tool_upsert_up.head(2))

,Id
0,a0y5w00000N0Um8AAF
1,a0y5w00000N0Um9AAF


,Name,Tool_Usage_Cluster__c,Tool__c
0,cntbands-ext_315,a0w5w000009Q7jnAAC,a0s5w00000k5MPqAAM
1,fettoy_315,a0w5w000009Q7jnAAC,a0s5w00000k5MQwAAM


In [64]:
db_tool_upsert.external_id = 'Id'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [65]:
db_tool_upsert.delete_data(db_tool_upsert_del)

[Success] Bulk job creation successful. Job ID = 7505w00000TpzMcAAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000TpzMcAAJ
[Success] Closing job successful. Job ID = 7505w00000TpzMcAAJ


In [66]:
db_tool_upsert.check_bulk_status()

{'id': '7505w00000TpzMcAAJ',
 'operation': 'delete',
 'object': 'ToolToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-09T18:58:58.000+0000',
 'systemModstamp': '2021-02-09T18:59:03.000+0000',
 'state': 'InProgress',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [67]:
pprint(db_tool_upsert.check_bulk_failed_results())

''


In [68]:
## now upload

In [69]:
db_tool_upsert.external_id = 'Name'
db_tool_upsert.object_id = 'ToolToolClusterAssociation__c'

In [70]:
db_tool_upsert.send_data(db_tool_upsert_up)

[Success] Bulk job creation successful. Job ID = 7505w00000TpzMmAAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000TpzMmAAJ
[Success] Closing job successful. Job ID = 7505w00000TpzMmAAJ


In [71]:
db_tool_upsert.check_bulk_status()

{'id': '7505w00000TpzMmAAJ',
 'operation': 'upsert',
 'object': 'ToolToolClusterAssociation__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-09T18:59:05.000+0000',
 'systemModstamp': '2021-02-09T18:59:09.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Name',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [72]:
pprint(db_tool_upsert.check_bulk_failed_results())

''


#### delete clustered tool cluster ids

In [73]:
db_del = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [74]:
db_del.object_id = 'tool_usage_cluster__c'
db_del.external_id = 'id'

In [75]:
db_del.delete_data(deletion_df)

[Success] Bulk job creation successful. Job ID = 7505w00000TpzN1AAJ
hello
[Success] CSV upload successful. Job ID = 7505w00000TpzN1AAJ
[Success] Closing job successful. Job ID = 7505w00000TpzN1AAJ


In [76]:
display(deletion_df.head(2))

,Id
0,a0w5w00000A8WKZAA3
1,a0w5w00000A8OJTAA3


In [77]:
db_del.check_bulk_status()

{'id': '7505w00000TpzN1AAJ',
 'operation': 'delete',
 'object': 'tool_usage_cluster__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-09T18:59:14.000+0000',
 'systemModstamp': '2021-02-09T18:59:17.000+0000',
 'state': 'InProgress',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [78]:
pprint(db_del.check_bulk_failed_results())

''


In [ ]:
### eventually, want to update DB2 with this information
